In [ ]:
# https://huggingface.co/MediaTek-Research/Llama-Breeze2-8B-Instruct

import torch
from transformers import AutoModel, AutoTokenizer
from transformers import GenerationConfig
from mtkresearch.llm.prompt import MRPromptV3

In [ ]:
model_id = 'MediaTek-Research/Llama-Breeze2-8B-Instruct-v0_1'

model = AutoModel.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map='auto',
    img_context_token_id=128212
).eval()

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)

generation_config = GenerationConfig(
  max_new_tokens=2048,
  do_sample=True,
  temperature=0.01,
  top_p=0.01,
  repetition_penalty=1.1,
  eos_token_id=128009
)

prompt_engine = MRPromptV3()

sys_prompt = 'You are a helpful AI assistant built by MediaTek Research. The user you are helping speaks Traditional Chinese and comes from Taiwan.'

def _inference(tokenizer, model, generation_config, prompt, pixel_values=None):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    if pixel_values is None:
        output_tensors = model.generate(**inputs, generation_config=generation_config)
    else:
        output_tensors = model.generate(**inputs, generation_config=generation_config, pixel_values=pixel_values.to(model.device, dtype=model.dtype))
    output_str = tokenizer.decode(output_tensors[0])
    return output_str

In [ ]:

def analyze_sentiment_llm(text):
    system_prompt = """
        # Role
        你是一位精通中文語意學與心理學的情感分析專家。你的任務是分析使用者提供的評論文本，並根據下方的「情緒類別學術定義」進行分類。

        # Objective
        1. 閱讀輸入的中文評論。
        2. 分析文本中的顯性情緒詞（Explicit Sentiment）與隱性情緒（Implicit Sentiment，如諷刺、語氣助詞）。
        3. 判斷該評論主要包含的一種或多種情緒。
        4. 嚴格按照定義輸出結果，不允許模糊地帶。

        # 情緒類別與學術定義
        請依據以下心理學與語言學定義進行判斷：

        1. **滿意**
        - **定義**：當個體評估當前情境與其目標一致，或需求得到滿足時產生的正向情緒。
        - **特徵**：包含滿意、興奮、愉悅。在評論中通常表現為對產品/服務的讚賞、推薦意願或感謝。

        2. **憤怒/不滿**
        - **定義**：當個體感知到目標受阻、受到不公待遇或期待落空時產生的激越情緒。
        - **特徵**：包含生氣、敵意、挫折感。在評論中常表現為指責、強烈的負面語氣助詞（如「爛透了」、「氣死」）、或攻擊性語言。

        3. **(悲傷/失望**
        - **定義**：當個體經歷不可挽回的損失（Loss）或無助感（Helplessness）時產生的低落情緒。
        - **特徵**：包含失落、沮喪、遺憾。在評論中表現為對結果的無力感、後悔購買、或委婉的抱怨（如「可惜了...」、「原本以為...」）。

        4. **恐懼/焦慮**
        - **定義**：當個體感知到潛在威脅、不確定性或無法掌控的情況時產生的防禦性情緒。
        - **特徵**：包含擔心、緊張、恐慌。在評論中表現為對產品安全性的疑慮、對售後服務的不確定感、或擔憂。

        5. **厭惡/反感**
        - **定義**：當個體接觸到被認為是有害、不潔、或違反社會規範的事物時產生的排斥反應。
        - **特徵**：包含噁心、鄙視、迴避。在評論中表現為對衛生狀況的批評、對服務態度的極度不屑、或生理上的不適感描述。

        6. **驚訝**
        - **定義**：當個體遭遇非預期（Unexpected）的事件時產生的短暫情緒狀態（可為正向或負向）。
        - **特徵**：包含震驚、錯愕。在評論中表現為「沒想到」、「竟然」、「出乎意料」。
        *注意：若驚訝後緊接著強烈的喜悅或憤怒，請同時標註。*

        7. **中性**
        - **定義**：缺乏明顯的情緒色彩，主要為客觀陳述事實或詢問資訊。
        - **特徵**：不帶個人主觀評價的描述。

        # Rules
        1. 若文本包含諷刺（Sarcasm），請依據其「真實意圖」而非字面意義進行分類（通常歸類為 Disgust 或 Anger）。
        2. 若文本包含多種情緒（Mixed Emotions），請列出所有主要情緒。
        3. 針對中文特有的語氣助詞（如：喔、吧、耶、蛤）需納入情緒強度的判斷依據。

        # Output Format
        請以 JSON 格式輸出，重點在於捕捉所有顯著的情緒：

        {
        "簡短分析": "簡短分析文本中的關鍵詞、語氣轉折與針對的對象...",
        "情緒": [
            {
            "情緒類別": "情緒類別 (使用中文)",
            "情緒強度": 1-5 (強度，5為最強烈),
            "該情緒針對的對象": "該情緒針對的對象 (例如: 服務, 產品, 物流)"
            }
        ]
    """

    user_prompt = f"評論內容：{text}"
    conversations = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    prompt = prompt_engine.get_prompt(conversations)
    output_str = _inference(tokenizer, model, generation_config, prompt)
    result = prompt_engine.parse_generated_str(output_str)
    return result

In [ ]:
# 測試
test_text_list = [
    "網路上有一家500元的吃到飽，大家的評論都很不錯，有各式各樣的菜色，但我實際去了一趟，發現全是便宜的澱粉類，肉片薄得像紙一樣，補菜還慢得要死。",
    "網路上有一家500元的吃到飽，大家的評論都很不錯，有各式各樣的菜色，今天去試了確實CP值很高，雖然海鮮不算頂級，但以500元這個價位來說，該有的都有，跟評論描述的差不多，沒讓人失望，是個粗飽的好選擇。",
    "網路上有一家500元的吃到飽，大家的評論都很不錯，有各式各樣的菜色，結果現場竟然有現切和牛，龍蝦，帝王蟹連甜點都是哈根達斯！",
    "網路上有一家500元的吃到飽，大家的評論都很不錯，有各式各樣的菜色，但是我只看得到炒飯和薯條，而且蒼蠅還幫忙加菜，這種『五星級』體驗真是讓我大開眼界，謝謝大家推薦喔。"
]

for text in test_text_list:
    result = analyze_sentiment_llm(text)
    print(f"輸入評論: {text}\n解析後的結果: {result}\n\n")